In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [2]:
# Necessary preprocessing
df = pd.read_csv('dataset/inhibitors.csv', encoding = 'latin1')
df = df.drop(columns = ['Name'])
df.fillna(df.mean(), inplace=True)
df = df.rename(columns = {'IC50 ': 'label'})
df['label'] = df['label'].apply(lambda x: (str(x))[:-2])
df['label'] = df['label'].apply(lambda x: x.strip())
df['label'] = df['label'].astype(float)
df.head()

,label,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,...,AMW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
0,30.0,0,2.0288,4.116029,37.5123,20.127930,0,0,17,7,...,6.708842,12.833789,1.833398,2.853553,0.000000,0.000000,56,4,2.011,22
1,340.0,0,-0.2231,0.049774,31.2928,19.874723,0,0,20,9,...,6.455072,16.327527,1.814170,12.922711,0.000000,12.922711,96,8,0.558,36
2,30.0,0,-1.4427,2.081383,40.0450,28.112688,0,0,27,11,...,5.782038,20.651188,1.877381,4.903390,4.903390,0.000000,200,9,2.160,40
3,36.3,1,-0.3390,0.114921,20.0274,23.580344,6,6,20,12,...,8.202367,23.394383,1.949532,7.309681,7.309681,0.000000,213,13,1.886,54
4,45.0,0,-0.5682,0.322851,12.2888,28.596723,10,11,24,13,...,7.211836,26.188765,2.014520,5.092296,2.543576,2.548720,218,21,2.369,68


In [3]:
features = list(df.columns)[1:]
print (features[:10])

['nAcid', 'ALogP', 'ALogp2', 'AMR', 'apol', 'naAromAtom', 'nAromBond', 'nAtom', 'nHeavyAtom', 'nH']


In [4]:
# Convert IC50 to categories
label_map = ["less than 1", "more than/equal to 1 and less than 30", "more than/equal to 30 and less than 60", "more than/equal to 60 and less than 100", "more than/equal to 100"]
for idx in df.index:
    if df['label'][idx] < 1:
        df['label'][idx] = 0
    elif df['label'][idx] >= 1 and df['label'][idx] < 30:
        df['label'][idx] = 1
    elif df['label'][idx] >=30 and df['label'][idx] < 60:
        df['label'][idx] = 2
    elif df['label'][idx] >= 60 and df['label'][idx] < 100:
        df['label'][idx] = 3
    elif df['label'][idx] >= 100:
        df['label'][idx] = 4
        
df['label'] = df['label'].astype(int)
df.head()

C:\Users\Yuvraj\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\Yuvraj\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\Yuvraj\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Yuvraj\Anaconda3\lib\site-package

,label,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,...,AMW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
0,2,0,2.0288,4.116029,37.5123,20.127930,0,0,17,7,...,6.708842,12.833789,1.833398,2.853553,0.000000,0.000000,56,4,2.011,22
1,4,0,-0.2231,0.049774,31.2928,19.874723,0,0,20,9,...,6.455072,16.327527,1.814170,12.922711,0.000000,12.922711,96,8,0.558,36
2,2,0,-1.4427,2.081383,40.0450,28.112688,0,0,27,11,...,5.782038,20.651188,1.877381,4.903390,4.903390,0.000000,200,9,2.160,40
3,2,1,-0.3390,0.114921,20.0274,23.580344,6,6,20,12,...,8.202367,23.394383,1.949532,7.309681,7.309681,0.000000,213,13,1.886,54
4,2,0,-0.5682,0.322851,12.2888,28.596723,10,11,24,13,...,7.211836,26.188765,2.014520,5.092296,2.543576,2.548720,218,21,2.369,68


In [5]:
# Necessary preprocessing
x = df.loc[:, features].values
x = StandardScaler().fit_transform(x)
pca = PCA(n_components = 10)   
principalComponents = pca.fit_transform(x)
pdf = pd.DataFrame(data = principalComponents)
df = pd.concat([pdf, df[['label']]], axis = 1)
features = list(df.columns)[:-1]

In [10]:
# # Uncomment this cell to re-train the model

# # Splitting data into test and train datasets
# train, test = train_test_split(df, test_size = 0.2)
# X_train, y_train, X_test, y_test = train[features], list(train['label']), test[features], list(test['label'])

# # Fit the model
# rf = RandomForestClassifier(max_depth = 5, max_features= 2, n_estimators = 15)
# rf = rf.fit(X_train, y_train)
# print ("Train accuracy : " + str(rf.score(X_train, y_train)))
# print ("Test accuracy : " + str(rf.score(X_test, y_test)))

Train accuracy : 0.9333333333333333
Test accuracy : 0.875


In [7]:
# Load trained model
import joblib

filename = 'models/inhibitors_model.sav'
rf = joblib.load(filename)

rf.get_params()

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 5,
 'max_features': 2,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 15,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [8]:
# Check model accuracy
X, y = df[features], list(df['label'])
print ("Overall model accuracy on entire (test + train) dataset : " + str(rf.score(X, y)))

Overall model accuracy on entire (test + train) dataset : 0.9210526315789473


In [9]:
# Predict AC 50 for new molecules
tdf = pd.read_csv('test/inhibitors_test.csv', encoding = 'latin1')
tdf = tdf.drop(columns = ['Name'])
tdf.fillna(0, inplace=True)
tx = tdf.values
tx = StandardScaler().fit_transform(tx)
tprincipalComponents = pca.transform(tx)
tpdf = pd.DataFrame(data = tprincipalComponents)
x_test = tpdf.values

preds = np.array(rf.predict(x_test))
for i in range(preds.size):
    print (str(i+1) + ". IC50 is " + label_map[preds[i]] + " um")

1. IC50 is more than/equal to 1 and less than 30 um
2. IC50 is more than/equal to 1 and less than 30 um
3. IC50 is more than/equal to 1 and less than 30 um
